In [1]:
%matplotlib inline
import glob
import os
import pandas as pd
import json
import numpy as np
import scipy.stats as ss
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
import matplotlib.pyplot as plt
import math
import warnings
warnings.simplefilter('ignore')


In [2]:
df = pd.read_csv("graphs/power_for_02-07-17_12H.csv")
df['in_power'] = df['in_power'].apply(pd.to_numeric)
df['out_power'] = df['out_power'].apply(pd.to_numeric)

In [3]:
df.head()

,in,in_amenity,in_power,influence,out,out_amenity,out_power
0,4037744278,bar,2.0,1.0,3186093217,restaurant,1.000000
1,4037744278,bar,2.0,1.0,404144840,parking,1.000000
2,404144840,parking,1.0,1.0,3648551975,restaurant,1.000000
3,3186093217,restaurant,1.0,1.0,3975094188,bar,1.000000
4,3648551975,restaurant,1.0,1.0,366700371,cafe,0.499287


In [5]:
df.sort_values(['in_power','influence'], ascending=False)

,in,in_amenity,in_power,influence,out,out_amenity,out_power
0,4037744278,bar,2.000000,1.000000,3186093217,restaurant,1.000000
1,4037744278,bar,2.000000,1.000000,404144840,parking,1.000000
27,3581990322,car_sharing,2.000000,1.000000,808930817,cafe,0.497522
28,3581990322,car_sharing,2.000000,1.000000,5276126259,restaurant,1.000000
38,4139840859,restaurant,2.000000,1.000000,4055481596,bar,1.000000
39,4139840859,restaurant,2.000000,1.000000,358805065,place_of_worship,1.000000
56,3976445299,bank,2.000000,1.000000,983388335,pharmacy,1.000000
57,3976445299,bank,2.000000,1.000000,3987038855,cafe,1.000000
35,2653086008,cafe,1.996382,1.000000,2538427496,pub,1.000000
36,2653086008,cafe,1.996382,0.996382,4139840859,restaurant,2.000000
